## ELT 

In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import boto3
import json
import time
from io import StringIO


In [2]:
# Configurar cliente de Athena y S3
athena_client = boto3.client("athena")
s3_client = boto3.client("s3")


In [3]:
# Base de datos en Glue
DATABASE = "econ"
S3_BUCKET = "itam-analytics-sofia"
QUERY_RESULTS = f"{S3_BUCKET}/query-results/"


In [4]:
create_db_query = f"CREATE DATABASE IF NOT EXISTS {DATABASE};"

athena_client.start_query_execution(
    QueryString=create_db_query,
    ResultConfiguration={"OutputLocation": f"s3://{QUERY_RESULTS}"}
)
print("✅ Base de datos creada en Glue: econ")


✅ Base de datos creada en Glue: econ


In [5]:
queries = [
    f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS {DATABASE}.tipo_de_cambio (
        date STRING,
        tipo_de_cambio DOUBLE
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
    WITH SERDEPROPERTIES ("skip.header.line.count" = "1")
    STORED AS TEXTFILE
    LOCATION 's3://{S3_BUCKET}/raw/tipo_de_cambio/';
    """,
    f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS {DATABASE}.tasa_de_interes (
        date STRING,
        tasa_de_interes DOUBLE
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
    WITH SERDEPROPERTIES ("skip.header.line.count" = "1")
    STORED AS TEXTFILE
    LOCATION 's3://{S3_BUCKET}/raw/tasa_de_interes/';
    """,
    f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS {DATABASE}.inflacion (
        date STRING,
        inpc DOUBLE,
        inflacion DOUBLE
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
    WITH SERDEPROPERTIES ("skip.header.line.count" = "1")
    STORED AS TEXTFILE
    LOCATION 's3://{S3_BUCKET}/raw/inflacion/';
    """
]


In [6]:
def ejecutar_query(query):
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={"Database": DATABASE},
        ResultConfiguration={"OutputLocation": f"s3://{QUERY_RESULTS}"}
    )
    return response

# Crear tablas en Athena
for query in queries:
    print("Ejecutando query...")
    response = ejecutar_query(query)
    query_id = response['QueryExecutionId']
    print(f"Query Execution ID: {query_id}")
    time.sleep(5)  # Pausa para evitar conflictos

print("✅ Tablas creadas en Athena.")


Ejecutando query...
Query Execution ID: 834d2e59-dc21-4a1e-ae66-230dce295dbb
Ejecutando query...
Query Execution ID: d49ab178-2832-4cf1-b7e8-7bf082177ed5
Ejecutando query...
Query Execution ID: 610efb77-b7fc-400a-a2d3-65bbfb89b5a6
✅ Tablas creadas en Athena.


In [7]:
create_economy_table = f"""
CREATE TABLE IF NOT EXISTS {DATABASE}.economy AS
SELECT 
    t.date,
    t.tasa_de_interes,
    i.inflacion,
    c.tipo_de_cambio
FROM {DATABASE}.tasa_de_interes t
JOIN {DATABASE}.inflacion i ON t.date = i.date
JOIN {DATABASE}.tipo_de_cambio c ON t.date = c.date;
"""

economy_response = ejecutar_query(create_economy_table)
print(f"Query Execution ID para economy: {economy_response['QueryExecutionId']}")
print("✅ Tabla economy creada en Athena.")


Query Execution ID para economy: 2133639d-14c1-4900-b69c-9310e5885a9c
✅ Tabla economy creada en Athena.


In [8]:
def descargar_datos():
    query = f"SELECT * FROM {DATABASE}.economy"
    response = ejecutar_query(query)
    query_id = response['QueryExecutionId']
    print(f"Descargando resultados de Query ID: {query_id}")
    
    # Esperar a que la consulta termine
    time.sleep(10)
    
    # Descargar el archivo desde S3
    result_file = f"query-results/{query_id}.csv"
    response = s3_client.get_object(Bucket=S3_BUCKET, Key=result_file)
    file_content = response["Body"].read().decode("utf-8")
    
    # Leer el contenido en un DataFrame
    df = pd.read_csv(StringIO(file_content))
    return df

# Descargar y mostrar los datos
df_economy = descargar_datos()
print(df_economy.head())


Descargando resultados de Query ID: 042d5d74-5d13-45d0-9403-84964b305d1c
         date  tasa_de_interes  inflacion  tipo_de_cambio
0  2016-01-01             3.08   2.613104       18.072775
1  2016-02-01             3.36   2.867251       18.473100
2  2016-03-01             3.80   2.601010       17.649045
3  2016-04-01             3.74   2.541579       17.487676
4  2016-05-01             3.81   2.596663       18.154186
